# Tutorial Básico de Gurobi para Programación Lineal

Este tutorial te enseñará los conceptos fundamentales para resolver problemas de programación lineal usando Gurobi en Python.

## Contenido:
1. Importar el módulo de Gurobi
2. Definir un problema ejemplo
3. Crear variables de decisión
4. Definir restricciones
5. Establecer la función objetivo
6. Resolver el problema
7. Extraer resultados y precios sombra


---

## 1. Importación del Módulo Gurobi

Primero debemos importar la librería de Gurobi. Asegúrate de tener Gurobi instalado y con una licencia válida.

In [3]:
import gurobipy as gp
from gurobipy import GRB

# También importamos otra librerías útil para hacer DataFrames
import pandas as pd

## 2. Definición del Problema Ejemplo

Vamos a resolver un problema clásico de programación lineal:

**Problema de Producción:**

Una empresa produce dos productos: A y B. Queremos maximizar las ganancias.

- Producto A: ganancia de $3 por unidad
- Producto B: ganancia de $5 por unidad

**Restricciones:**
- Tiempo de máquina: 2 horas para A, 1 hora para B (máximo 100 horas disponibles)
- Material: 1 kg para A, 3 kg para B (máximo 80 kg disponibles)
- Demanda mínima: al menos 5 unidades de A
- Variables no negativas

**Formulación matemática:**

Maximizar: 3x₁ + 5x₂

Sujeto a:
- 2x₁ + x₂ ≤ 100  (tiempo de máquina)
- x₁ + 3x₂ ≤ 80   (material)
- x₁ ≥ 5          (demanda mínima)
- x₁, x₂ ≥ 0      (no negatividad)

## 3. Creación del Modelo

En Gurobi, todo problema se define dentro de un "modelo". Primero creamos el modelo:

In [4]:
# Crear el modelo
modelo = gp.Model("ProblemaProduccion")

Set parameter Username
Set parameter LicenseID to value 2596460
Academic license - for non-commercial use only - expires 2025-12-06
Set parameter LicenseID to value 2596460
Academic license - for non-commercial use only - expires 2025-12-06


## 4. Definición de Variables de Decisión

Las variables de decisión representan las cantidades que queremos determinar. En nuestro caso, las cantidades a producir de cada producto.

Pese a ser positivas irrestrictas las variables, es buena práctica utilizar un número muy grande positivo como límite superior para mejorar la convergencia. Se denomina comúnmente `bigM` a este número muy grande. Para este ejemplo será 1 millón.

En este ejemplo las variables son continuas, por lo que su tipo será `GRB.CONTINUOUS`. Sin embargo, gurobi también permite definir otros tipos de variables, como binarias (`GRB.BINARY`) o enteras (`GRB.INTEGER`).

In [11]:
# Definir variables de decisión
# addVar(lb=lower_bound, ub=upper_bound, vtype=variable_type, name="nombre")

x1 = modelo.addVar(lb=0, ub=1e6, vtype=GRB.CONTINUOUS, name="ProductoA") # Una variable puede ser Continuous, Binary o Integer
x2 = modelo.addVar(lb=0, ub=1e6, vtype=GRB.CONTINUOUS, name="ProductoB")

# Actualizar el modelo para que reconozca las nuevas variables
modelo.update()

## 5. Definición de Restricciones

Las restricciones limitan el espacio de soluciones factibles. Se definen usando `addConstr()`.

In [12]:
# Definir restricciones

# Restricción 1: Tiempo de máquina (2x1 + x2 <= 100)
restriccion_tiempo = modelo.addConstr(2*x1 + x2 <= 100, name="TiempoMaquina")

# Restricción 2: Material (x1 + 3x2 <= 80)
restriccion_material = modelo.addConstr(x1 + 3*x2 <= 80, name="Material")

# Restricción 3: Demanda mínima (x1 >= 5)
restriccion_demanda = modelo.addConstr(x1 >= 5, name="DemandaMinima")

Las restricciones de no negatividad $(x1, x2 >= 0)$ ya están incluidas en la definición de las variables con $lb=0$

## 6. Definición de la Función Objetivo

La función objetivo define qué queremos optimizar (maximizar o minimizar).

In [14]:
# Definir función objetivo
# Maximizar: 3x1 + 5x2
modelo.setObjective(3*x1 + 5*x2, GRB.MAXIMIZE)

## 7. Resolución del Problema

Una vez definido completamente el modelo, usamos `optimize()` para resolverlo.

In [16]:
# Resolver el problema
modelo.optimize()

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 3 rows, 10 columns and 5 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [1e+06, 1e+06]
  RHS range        [5e+00, 1e+02]

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.920000000e+02


## 8. Extracción de Resultados

Si el problema tiene solución óptima, podemos extraer los valores de las variables y el valor objetivo.

In [17]:
# Extraer resultados si la solución es óptima
if modelo.status == GRB.OPTIMAL:
    print("=== RESULTADOS DE LA OPTIMIZACIÓN ===")
    print(f"Valor objetivo óptimo: ${modelo.objVal:.2f}")
    print()
    
    # Valores de las variables
    print("Valores óptimos de las variables:")
    print(f"Producto A (x1): {x1.x:.2f} unidades")
    print(f"Producto B (x2): {x2.x:.2f} unidades")
    print()
    
    # Verificar qué restricciones están activas (binding)
    print("Estado de las restricciones:")
    print(f"Tiempo de máquina: {2*x1.x + x2.x:.2f} / 100 horas")
    print(f"Material: {x1.x + 3*x2.x:.2f} / 80 kg")
    print(f"Demanda mínima: {x1.x:.2f} / 5 unidades mínimas")
else:
    print("No se pudo obtener una solución óptima")

=== RESULTADOS DE LA OPTIMIZACIÓN ===
Valor objetivo óptimo: $192.00

Valores óptimos de las variables:
Producto A (x1): 44.00 unidades
Producto B (x2): 12.00 unidades

Estado de las restricciones:
Tiempo de máquina: 100.00 / 100 horas
Material: 80.00 / 80 kg
Demanda mínima: 44.00 / 5 unidades mínimas


## 9. Extracción de Precios Sombra (Dual Values)

Los precios sombra nos indican cuánto aumentaría el valor objetivo si relajáramos cada restricción en una unidad.

In [20]:
# Extraer precios sombra (valores duales)
if modelo.status == GRB.OPTIMAL:
    print("=== PRECIOS SOMBRA (VALORES DUALES) ===")
    print()
    
    # Precios sombra de las restricciones
    print(f"Tiempo de máquina: ${restriccion_tiempo.pi:.2f} por hora adicional")
    print(f"Material: ${restriccion_material.pi:.2f} por kg adicional")
    print(f"Demanda mínima: ${restriccion_demanda.pi:.2f} por unidad adicional requerida")
    print()

=== PRECIOS SOMBRA (VALORES DUALES) ===

Tiempo de máquina: $0.80 por hora adicional
Material: $1.40 por kg adicional
Demanda mínima: $0.00 por unidad adicional requerida



## 10. Conceptos Clave Aprendidos

### Proceso paso a paso en Gurobi:

1. **Importar la librería:** `import gurobipy as gp`
2. **Crear modelo:** `modelo = gp.Model("nombre")`
3. **Definir variables:** `x = modelo.addVar(lb=0, vtype=GRB.CONTINUOUS, name="x")`
4. **Agregar restricciones:** `modelo.addConstr(expresión, name="nombre")`
5. **Establecer objetivo:** `modelo.setObjective(expresión, GRB.MAXIMIZE)`
6. **Resolver:** `modelo.optimize()`
7. **Extraer resultados:** `variable.x` para valores, `restriccion.pi` para precios sombra

## 11. Ejercicio Práctico

### Problema para resolver:

Una panadería produce dos tipos de pan: integral y blanco.

**Datos:**
- Pan integral: ganancia de $2 por unidad
- Pan blanco: ganancia de $1.5 por unidad

**Restricciones:**
- Horno: 3 horas para integral, 2 horas para blanco (máximo 120 horas/día)
- Harina integral: 1 kg por unidad de pan integral (máximo 25 kg/día)
- Harina común: 0.5 kg por unidad de pan blanco (máximo 30 kg/día)
- Demanda mínima: al menos 10 panes integrales por día

Resuelve este problema usando los comandos aprendidos de Gurobipy